# Gene ontology analysis

In [1]:
import pandas as pd

## Download ontologies

Download ontologies, a dictionary that maps GO IDs to GO terms. In most cases, we should use the basic OBO file.

In [2]:
from goatools.base import download_go_basic_obo
from goatools.obo_parser import GODag

obo_fname = download_go_basic_obo()
ontologies = GODag(obo_fname)

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2019-07-01) 47,413 GO Terms


In [3]:
first = list(ontologies.keys())[0]
first, ontologies[first]

('GO:0000001', GOTerm('GO:0000001'):
   id:GO:0000001
   item_id:GO:0000001
   name:mitochondrion inheritance
   namespace:biological_process
   _parents: 2 items
     GO:0048311
     GO:0048308
   parents: 2 items
     GO:0048308	level-04	depth-04	organelle inheritance [biological_process]
     GO:0048311	level-05	depth-05	mitochondrion distribution [biological_process]
   children: 0 items
   level:5
   depth:6
   is_obsolete:False
   alt_ids: 0 items)

## Download associations

Download associations, a dictionary that maps each gene ID to a set of GOs.
We can use either the associations from NCBI or from GeneOntology.

In [4]:
# from goatools.base import download_ncbi_associations
# from goatools.anno.genetogo_reader import Gene2GoReader

# # Read NCBI's gene2go. Store annotations in a list of namedtuples
# file_gene2go = download_ncbi_associations()
# taxids = [7955] # zebrafish
# objanno = Gene2GoReader(file_gene2go, taxids=taxids)

# associations = objanno.get_ns2assc()
# for nspc, id2gos in associations.items():
#     print("{NS} {N:,} annotated genes".format(NS=nspc, N=len(id2gos)))

In [5]:
from goatools.associations import read_gaf
from goatools.base import dnld_gaf

# see http://current.geneontology.org/products/pages/downloads.html
species = 'zfin' # choices are 'goa_human', 'mgi' etc
NS = 'BP' # choices are 'BP', 'CC' or 'MF'

gaf_filename = dnld_gaf(species)
associations = read_gaf(gaf_filename, namespace=NS, go2geneids=False)
print("{N:,} annotated genes".format(N=len(associations)))

HMS:0:00:04.273083 227,647 annotations READ: C:\Users\joewa\Work\git\WebOmics\web_omics\notebooks\gene_ontology\zfin.gaf 
17785 IDs in loaded association branch, BP
17,785 annotated genes


In [6]:
first = list(associations.keys())[0]
first, associations[first]

('ZDB-MIRNAG-081210-6', {'GO:0030182', 'GO:0035195'})

## Load background genes

In [7]:
def gaf_symbol_to_id(gaf_filename):
    df = pd.read_csv(gaf_filename, comment='!', sep='\t', header=None)
    
    # temp has 2 columns. First is the gene id, next is the gene symbol
    # example:
    # 'ZDB-MIRNAG-081210-6', 'mir26b'
    temp = df.iloc[:, 1:3].values 
    symbol_to_id = {symbol: my_id for my_id, symbol in temp}
    return symbol_to_id

In [8]:
def load_background_symbols(symbol_filename):
    df = pd.read_csv(symbol_filename, header=None)
    background_symbols = df.values.flatten()
    return background_symbols

In [9]:
def to_id(symbols):
    ids = []
    for x in symbols:
        try:
            my_id = symbol_to_id[x.lower()]
            ids.append(my_id)
        except KeyError as e:
            # print(e)
            pass
    return ids

In [10]:
symbol_to_id = gaf_symbol_to_id('zfin.gaf')

c:\users\joewa\.virtualenvs\web_omics-makc_z5x\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [11]:
# here we use all genes in the study as the background, see https://www.biostars.org/p/17628/
background_symbols = load_background_symbols('zebrafish.txt')
background_ids = to_id(background_symbols)
len(background_ids)

8242

#### Initialise GOEA object

In [12]:
from goatools.go_enrichment import GOEnrichmentStudy
goeaobj = GOEnrichmentStudy(
        background_ids,
        associations,
        ontologies, 
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


Load GOEA Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 89%  7,295 of  8,242 population items found in association


In [13]:
geneids_study = background_ids[0:400]
geneids_study

['ZDB-GENE-060824-3',
 'ZDB-GENE-030912-4',
 'ZDB-GENE-040426-903',
 'ZDB-GENE-130530-713',
 'E7F5G8',
 'ZDB-GENE-081105-101',
 'ZDB-GENE-091118-25',
 'ZDB-GENE-040329-1',
 'ZDB-GENE-991019-6',
 'ZDB-GENE-050913-36',
 'ZDB-GENE-061220-8',
 'ZDB-GENE-031006-4',
 'ZDB-GENE-030131-9790',
 'ZDB-GENE-031006-12',
 'ZDB-GENE-050517-1',
 'ZDB-GENE-050517-2',
 'ZDB-GENE-050517-3',
 'ZDB-GENE-050517-4',
 'ZDB-GENE-050517-5',
 'ZDB-GENE-040525-2',
 'ZDB-GENE-050517-14',
 'ZDB-GENE-080204-52',
 'ZDB-GENE-050517-9',
 'ZDB-GENE-070912-584',
 'ZDB-GENE-050517-10',
 'ZDB-GENE-050410-6',
 'ZDB-GENE-050517-12',
 'ZDB-GENE-050517-15',
 'ZDB-GENE-050517-25',
 'ZDB-GENE-040426-1523',
 'ZDB-GENE-050517-16',
 'ZDB-GENE-030616-511',
 'ZDB-GENE-050517-17',
 'ZDB-GENE-050517-22',
 'E7F108',
 'ZDB-GENE-050517-23',
 'ZDB-GENE-050517-27',
 'ZDB-GENE-050517-28',
 'ZDB-GENE-040426-2868',
 'ZDB-GENE-050517-29',
 'ZDB-GENE-050517-30',
 'ZDB-GENE-050517-31',
 'ZDB-GENE-070424-84',
 'ZDB-GENE-050517-35',
 'ZDB-GENE-0505

#### Load actual study genes from the Zebrafish paper

In [21]:
df = pd.read_pickle('C:\\Users\\joewa\\Work\\git\\WebOmics\\web_omics\\notebooks\\gene_ontology\\selection_df.p')
df.head()

,obs,gene_pk,US_1584693,US_1584700,US_1584706,US_1584712,US_1584722,US_1584724,US_1584725,US_1584732,...,US_1584753,US_1584754,US_1584758,US_1584765,padj_Distal_vs_Middle,FC_Distal_vs_Middle,significant_all,significant_any,padj_Proximal_vs_Middle,FC_Proximal_vs_Middle
gene_id,,,,,,,,,,,,,,,,,,,,,
Ptpra,True,ENSDARG00000001769,2534.0,1967.0,1966.0,1793.0,1557.0,2355.0,2451.0,2344.0,...,2521.0,1741.0,2689.0,2092.0,0.564925,0.063288,False,False,0.228854,-0.140858
C3b.2,True,ENSDARG00000001818,1.0,1.0,1.0,0.0,0.0,3.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.061724,2.901142,False,False,NaN,-0.341172
Aplnra,True,ENSDARG00000002172,8.0,7.0,14.0,12.0,22.0,13.0,17.0,10.0,...,7.0,18.0,13.0,15.0,0.006370,-1.095306,False,True,0.880153,-0.119547
Adora2b,True,ENSDARG00000002533,171.0,142.0,173.0,104.0,123.0,139.0,159.0,137.0,...,129.0,135.0,160.0,129.0,0.803429,-0.047145,False,False,0.846411,-0.049721
Gnb3b,True,ENSDARG00000002696,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,...,2.0,1.0,1.0,0.0,0.905430,-0.210661,False,False,NaN,0.563356


In [22]:
genesymbols_study = df.index.values
geneids_study = to_id(genesymbols_study)
len(geneids_study)

678

#### Run GO Analysis

In [23]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
goea_results_all = goeaobj.run_study(geneids_study)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run GOEA Gene Ontology Analysis: current study set of 678 IDs ...
 92%    565 of    613 study items found in association
 90%    613 of    678 study items found in population(8242)
Calculating 4,226 uncorrected p-values using fisher_scipy_stats
   4,226 GO terms are associated with  7,230 of  8,242 population items
     771 GO terms are associated with    565 of    645 study items
  METHOD fdr_bh:
     113 GO terms found significant (< 0.05=alpha) (105 enriched +   8 purified): statsmodels fdr_bh
     445 study items associated with significant GO IDs (enriched)
      16 study items associated with significant GO IDs (purified)


In [24]:
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj
plot_results("zebrafish_{NS}.png", goea_results_sig)

  113 usr 640 GOs  WROTE: zebrafish_BP.png


In [25]:
goea_results_sig

[GOEnrichmentRecord(GO:0007186),
 GOEnrichmentRecord(GO:0007165),
 GOEnrichmentRecord(GO:0007218),
 GOEnrichmentRecord(GO:0006935),
 GOEnrichmentRecord(GO:0030198),
 GOEnrichmentRecord(GO:0008543),
 GOEnrichmentRecord(GO:0019722),
 GOEnrichmentRecord(GO:0007204),
 GOEnrichmentRecord(GO:0070098),
 GOEnrichmentRecord(GO:0060326),
 GOEnrichmentRecord(GO:0006955),
 GOEnrichmentRecord(GO:0007187),
 GOEnrichmentRecord(GO:0006954),
 GOEnrichmentRecord(GO:0007188),
 GOEnrichmentRecord(GO:0071880),
 GOEnrichmentRecord(GO:0070374),
 GOEnrichmentRecord(GO:0007189),
 GOEnrichmentRecord(GO:0008284),
 GOEnrichmentRecord(GO:0030593),
 GOEnrichmentRecord(GO:0007169),
 GOEnrichmentRecord(GO:0007268),
 GOEnrichmentRecord(GO:0060079),
 GOEnrichmentRecord(GO:0030595),
 GOEnrichmentRecord(GO:0007166),
 GOEnrichmentRecord(GO:0061844),
 GOEnrichmentRecord(GO:0007399),
 GOEnrichmentRecord(GO:0043410),
 GOEnrichmentRecord(GO:0071346),
 GOEnrichmentRecord(GO:0060012),
 GOEnrichmentRecord(GO:0015718),
 GOEnrichm

In [26]:
goea_results_sig[0]

GOEnrichmentRecord(GO:0007186)

In [39]:
id_to_symbol = {v: k for k, v in symbol_to_id.items()}

In [48]:
data = []
for record in goea_results_sig:
    study_ratio = record.ratio_in_study[0] / record.ratio_in_study[1]
    pop_ratio = record.ratio_in_pop[0] / record.ratio_in_pop[1]
    study_names = list(map(lambda x: id_to_symbol[x], record.study_items))
    row = [
        record.GO, record.name, 
        study_ratio, pop_ratio, 
        record.get_pvalue(), record.depth, 
        record.study_count, study_names
    ]
    data.append(row)

In [49]:
df = pd.DataFrame(data, columns=[
    'GO', 'name', 
    'ratio_in_study', 'ratio_in_pop', 
    'pvalue', 'depth', 
    'study_count', 'study_items'
])

In [52]:
df = df.set_index('GO')

In [54]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name</th>\n      <th>ratio_in_study</th>\n      <th>ratio_in_pop</th>\n      <th>pvalue</th>\n      <th>depth</th>\n      <th>study_count</th>\n      <th>study_items</th>\n    </tr>\n    <tr>\n      <th>GO</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>GO:0007186</th>\n      <td>G protein-coupled receptor signaling pathway</td>\n      <td>0.319739</td>\n      <td>0.040281</td>\n      <td>1.761663e-139</td>\n      <td>5</td>\n      <td>196</td>\n      <td>[oxgr1a.2, cxcr4b, htr1d, gpr176, chrm4a, grm8b, htr1fa, adrb3a, ptger2b, s1pr2, htr1b, p2ry12, fshr, si:dkey-44g17.6, taar12j, ccr2, aplnrb, taar18d, oprm1, ccr6a, ccr9a, cnr2, adrb3b, lpar3, ptger3, drd2b, npy2r, calcrlb, oprd1b, mc2r, grm4, ackr3b, mc5rb, lpar1, sstr3, gng13b, gng10, ccr7, g